<a href="https://colab.research.google.com/github/AndersieMichael/Thesis/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
import pandas as pd
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
# Ganti 'path_to_your_dataset' dengan path aktual ke folder dataset di Google Drive Anda
# path_to_dataset = '/content/drive/MyDrive/Colab Notebooks/Deep Learning/test skripsi/dataset/dataMerge'
path_to_dataset = '/content/drive/MyDrive/Colab/deep learning/dataset/dataMerge'
train_dir = os.path.join(path_to_dataset, 'train')
test_dir = os.path.join(path_to_dataset, 'test')
# val_dir = os.path.join(path_to_dataset, 'val')

# Contoh: Akses dan list file dalam direktori dataset

files = os.listdir(path_to_dataset)
print(files)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['test', 'train', 'new_folder', 'new_folder2', 'train.cache', 'test.cache']


In [ ]:
#initialisasi variable

mystate =32
imageSize = 224
epochs = 50
batch_size = 64
learningRate=0.001
weight_decay=0.0005

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
from torchvision.io import read_image
from torchvision.transforms.functional import convert_image_dtype


In [ ]:
import torchvision.datasets as datasets

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
# Periksa distribusi kelas
class_counts = {}
for _, label in train_dataset:
    if label in class_counts:
        class_counts[label] += 1
    else:
        class_counts[label] = 1

class_names = train_dataset.classes

# Cetak informasi distribusi kelas
for class_name, count in zip(class_names, class_counts.values()):
    print(f"Class: {class_name}, Number of images: {count}")

Class: Acne, Number of images: 138
Class: Actinic Keratosis, Number of images: 138
Class: Basal Cell Carcinoma, Number of images: 138
Class: Eczemaa, Number of images: 138
Class: Rosacea, Number of images: 138


In [ ]:
# # Dapatkan daftar semua subdirektori di dalam train_dir
# train_class_names = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]
# num_classes = len(train_class_names)

# print("Jumlah kelas:", num_classes)

In [ ]:
def optimizer_Setting(model):
        return torch.optim.Adam(model.parameters(), lr=learningRate,  weight_decay = weight_decay)


In [ ]:
from torchvision.models import mobilenet_v2

# Load pretrained MobileNet V2 model
model = mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(class_names))  # Sesuaikan layer terakhir

# Membekukan layer terakhir (classifier)
for param in model.classifier.parameters():
    param.requires_grad = False

# Membuka freeze untuk layer lainnya
for name, param in model.named_parameters():
    if "classifier" not in name:
        param.requires_grad = True

# Kriteria dan optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learningRate,  weight_decay = weight_decay)
optimizer = optimizer_Setting(model)

# Tentukan device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 101MB/s] 


In [ ]:
# Tentukan path penyimpanan model
save_path = 'best_model.pth'

# Early stopping dan saving best model
best_loss = float('inf')
patience = 5
trigger_times = 0

In [ ]:
# Fungsi pelatihan
def train_model(model, criterion, optimizer, train_loader, test_loader,  save_path, num_epochs=15):
   # Early stopping dan saving best model
    best_loss = float('inf')
    patience = 5
    trigger_times = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backpropagation dan optimasi
            loss.backward()
            optimizer.step()

            # Statistik
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = correct / total
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validasi dan cek early stopping
        val_loss = validate(model, test_loader)
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), save_path)
            print(f'Best model saved with loss: {best_loss:.4f}')
            trigger_times = 0
        else:
            trigger_times += 1
            print(f'No improvement in loss for {trigger_times} epochs')

        if trigger_times >= patience:
            print('Early stopping triggered.')
            break


In [ ]:
def validate(model, test_loader):
    model.eval()
    running_loss = 0.0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

    return running_loss / len(test_dataset)

In [ ]:

mobileNetV2_save_path = '/content/drive/MyDrive/Colab/deep learning/mobileNetV2_best_model.pth'

# Pelatihan dan pengujian model
train_model(model, criterion, optimizer, train_loader, test_loader, mobileNetV2_save_path, num_epochs=epochs)

Epoch 1/50, Loss: 1.1103, Accuracy: 0.5652
Best model saved with loss: 2.7768
Epoch 2/50, Loss: 0.6339, Accuracy: 0.7667
Best model saved with loss: 1.0904
Epoch 3/50, Loss: 0.4008, Accuracy: 0.8551
Best model saved with loss: 0.9163
Epoch 4/50, Loss: 0.2837, Accuracy: 0.9029
Best model saved with loss: 0.8065
Epoch 5/50, Loss: 0.2063, Accuracy: 0.9275
No improvement in loss for 1 epochs
Epoch 6/50, Loss: 0.1895, Accuracy: 0.9377
Best model saved with loss: 0.8059
Epoch 7/50, Loss: 0.1165, Accuracy: 0.9681
Best model saved with loss: 0.6358
Epoch 8/50, Loss: 0.1090, Accuracy: 0.9638
No improvement in loss for 1 epochs
Epoch 9/50, Loss: 0.1331, Accuracy: 0.9609
No improvement in loss for 2 epochs
Epoch 10/50, Loss: 0.1040, Accuracy: 0.9667
No improvement in loss for 3 epochs
Epoch 11/50, Loss: 0.0816, Accuracy: 0.9696
Best model saved with loss: 0.6287
Epoch 12/50, Loss: 0.0535, Accuracy: 0.9826
No improvement in loss for 1 epochs
Epoch 13/50, Loss: 0.0610, Accuracy: 0.9812
No improveme

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Hitung akurasi
    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Accuracy: {accuracy:.4f}')


In [ ]:
# Load model terbaik yang sudah disimpan
model.load_state_dict(torch.load(mobileNetV2_save_path))

# Evaluasi model
evaluate(model, test_loader )

<ipython-input-27-ff2755d9c3f3>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(mobileNetV2_save_path))


Accuracy: 0.8000


In [ ]:
from torchvision.models import mobilenet_v3_small

# Load pretrained MobileNet V3 Small model
model_v3_small = mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model_v3_small.classifier[3] = nn.Linear(model_v3_small.classifier[3].in_features,  len(class_names))  # Sesuaikan layer terakhir

# Membekukan layer terakhir (classifier)
for param in model_v3_small.classifier.parameters():
    param.requires_grad = False

# Membuka freeze untuk layer lainnya
for name, param in model_v3_small.named_parameters():
    if "classifier" not in name:
        param.requires_grad = True


# Kriteria dan optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model_v3_small.parameters(), lr=learningRate)
optimizer = optimizer_Setting(model_v3_small)

# Tentukan device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_v3_small = model_v3_small.to(device)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 17.4MB/s]


In [ ]:
mobileNetV3_small_save_path = '/content/drive/MyDrive/Colab/deep learning/mobileNetV3_small_best_model.pth'

# Pelatihan dan pengujian model
train_model(model_v3_small, criterion, optimizer, train_loader, test_loader, mobileNetV3_small_save_path, num_epochs=epochs)

Epoch 1/50, Loss: 1.3638, Accuracy: 0.4594
Best model saved with loss: 1.5321
Epoch 2/50, Loss: 0.8693, Accuracy: 0.6855
No improvement in loss for 1 epochs
Epoch 3/50, Loss: 0.5814, Accuracy: 0.8145
Best model saved with loss: 1.2519
Epoch 4/50, Loss: 0.3720, Accuracy: 0.8928
No improvement in loss for 1 epochs
Epoch 5/50, Loss: 0.2308, Accuracy: 0.9362
No improvement in loss for 2 epochs
Epoch 6/50, Loss: 0.1201, Accuracy: 0.9710
Best model saved with loss: 1.1993
Epoch 7/50, Loss: 0.0626, Accuracy: 0.9899
No improvement in loss for 1 epochs
Epoch 8/50, Loss: 0.0459, Accuracy: 0.9913
No improvement in loss for 2 epochs
Epoch 9/50, Loss: 0.0654, Accuracy: 0.9783
No improvement in loss for 3 epochs
Epoch 10/50, Loss: 0.1116, Accuracy: 0.9652
No improvement in loss for 4 epochs
Epoch 11/50, Loss: 0.1228, Accuracy: 0.9623
No improvement in loss for 5 epochs
Early stopping triggered.


In [ ]:
from torchvision.models import mobilenet_v3_large

# Load pretrained MobileNet V3 large model
model_v3_large = mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)
model_v3_large.classifier[3] = nn.Linear(model_v3_large.classifier[3].in_features,  len(class_names))  # Sesuaikan layer terakhir

# Membekukan layer terakhir (classifier)
for param in model_v3_large.classifier.parameters():
    param.requires_grad = False

# Membuka freeze untuk layer lainnya
for name, param in model_v3_large.named_parameters():
    if "classifier" not in name:
        param.requires_grad = True
# Kriteria dan optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model_v3_large.parameters(), lr=learningRate)
optimizer = optimizer_Setting(model_v3_large)

# Tentukan device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_v3_large = model_v3_large.to(device)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 94.7MB/s]


In [ ]:
# mobileNetV3_large_save_path = '/content/drive/MyDrive/Colab Notebooks/Deep Learning/test skripsi/pytorch code/mobileNetV3_large_best_model.pth'
mobileNetV3_large_save_path = '/content/drive/MyDrive/Colab/deep learning/mobileNetV3_large_best_model.pth'
# Pelatihan dan pengujian model
train_model(model_v3_large, criterion, optimizer, train_loader, test_loader, mobileNetV3_large_save_path, num_epochs=epochs)

Epoch 1/50, Loss: 1.1604, Accuracy: 0.5536
Best model saved with loss: 1.1575
Epoch 2/50, Loss: 0.5428, Accuracy: 0.8145
No improvement in loss for 1 epochs
Epoch 3/50, Loss: 0.2505, Accuracy: 0.9319
No improvement in loss for 2 epochs
Epoch 4/50, Loss: 0.1142, Accuracy: 0.9652
No improvement in loss for 3 epochs
Epoch 5/50, Loss: 0.0855, Accuracy: 0.9826
No improvement in loss for 4 epochs
Epoch 6/50, Loss: 0.0764, Accuracy: 0.9783
Best model saved with loss: 1.0969
Epoch 7/50, Loss: 0.0651, Accuracy: 0.9797
No improvement in loss for 1 epochs
Epoch 8/50, Loss: 0.0829, Accuracy: 0.9768
No improvement in loss for 2 epochs
Epoch 9/50, Loss: 0.0464, Accuracy: 0.9913
No improvement in loss for 3 epochs
Epoch 10/50, Loss: 0.0360, Accuracy: 0.9928
No improvement in loss for 4 epochs
Epoch 11/50, Loss: 0.0528, Accuracy: 0.9899
No improvement in loss for 5 epochs
Early stopping triggered.


In [ ]:
from torchvision.models import efficientnet_b2, EfficientNet_B2_Weights

# Load pretrained EfficientNet B2 model
model_efficientnet_b2 = efficientnet_b2(weights=EfficientNet_B2_Weights.IMAGENET1K_V1)
model_efficientnet_b2.classifier[1] = nn.Linear(model_efficientnet_b2.classifier[1].in_features,  len(class_names))

# Membekukan layer terakhir (classifier)
for param in model_efficientnet_b2.classifier.parameters():
    param.requires_grad = False

# Membuka freeze untuk layer lainnya
for name, param in model_efficientnet_b2.named_parameters():
    if "classifier" not in name:
        param.requires_grad = True

# Kriteria dan optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model_efficientnet_b2.parameters(), lr=learningRate)
optimizer = optimizer_Setting(model_efficientnet_b2)

# Tentukan device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_efficientnet_b2 = model_efficientnet_b2.to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:01<00:00, 21.9MB/s]


In [ ]:
# efficientNet_b2_save_path = '/content/drive/MyDrive/Colab Notebooks/Deep Learning/test skripsi/pytorch code/efficientNet_b2_best_model.pth'
efficientNet_b2_save_path = '/content/drive/MyDrive/Colab/deep learning/efficientNet_b2_best_model.pth'
# Pelatihan dan pengujian model
train_model(model_efficientnet_b2, criterion, optimizer, train_loader, test_loader, efficientNet_b2_save_path, num_epochs=epochs)

Epoch 1/50, Loss: 1.1651, Accuracy: 0.5420
Best model saved with loss: 1.1484
Epoch 2/50, Loss: 0.4792, Accuracy: 0.8493
Best model saved with loss: 0.6716
Epoch 3/50, Loss: 0.1952, Accuracy: 0.9478
Best model saved with loss: 0.4511
Epoch 4/50, Loss: 0.1206, Accuracy: 0.9609
No improvement in loss for 1 epochs
Epoch 5/50, Loss: 0.1033, Accuracy: 0.9652
No improvement in loss for 2 epochs
Epoch 6/50, Loss: 0.1470, Accuracy: 0.9609
No improvement in loss for 3 epochs
Epoch 7/50, Loss: 0.1395, Accuracy: 0.9565
No improvement in loss for 4 epochs
Epoch 8/50, Loss: 0.1305, Accuracy: 0.9565
No improvement in loss for 5 epochs
Early stopping triggered.
